In [7]:
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv
import openai
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

In [8]:
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4-turbo")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)

# Memória resumida para manter o contexto sem gastar muitos tokens
memory = ConversationSummaryMemory(llm=llm)

In [3]:
Arandom = Agent(
    role="Gerador de conceitos aleatorias",
    goal="Sua tarefa será falar palavras aleatórias, como animais, pessoas, paises, objetos, etc",
    backstory=f"Você deve gerar palavras aleatórias para que o contador de historias possa adicionar em sua historia",
    llm=llm,
    memory=memory
)

Ahistory = Agent(
    role="Contador de historas",
    goal="Sua tarefa será ouvir as palavras aleatórias do gerador de conceitos e criar histórias a partir deles, adicionando os elementos todos a uma mesma história",
    backstory="Você cria histórias, e a partir de conceitos aleatórios deve criar e adicionar esses conceitos em suas histórias",
    llm=llm,
    memory=memory
)

def GetHistoryAIResponse(InputText):
    taskHisotry = Task(
    description=(
        f"Você deve adicionar o seguinte contexto em sua história: {InputText}"
    ),
    agent=Ahistory, 
    memory=memory,
    expected_output="Gere uma história somando os contextos apresentados a sua história, caso ainda não haja história comece com o contexto apresentado" 
    )

    crewBuyer = Crew(
    agents=[Ahistory],
    tasks=[taskHisotry]
    )

    return crewBuyer.kickoff()

def GetRandomAIResponse():
    taskRandom = Task(
    description=(
        "Retorne uma palavra completamente aleatoria, podendo ser um pais, animal, objeto ou qualquer outra coisa, nunca retorne a mesma palavra duas vezes"
    ),
    agent=Arandom, 
    memory=memory,
    expected_output="Apenas uma palavra aleatória"
    )

    crewSeller = Crew(
    agents=[Arandom],
    tasks=[taskRandom]
    )

    return crewSeller.kickoff()

In [ ]:
# Carregar variáveis do .env
load_dotenv()

# Usar as variáveis carregadas
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)

# Simulação do diálogo
RandomSpeaking = True

for i in range(6):
    if RandomSpeaking:
        resultado = GetRandomAIResponse()
        print(resultado)
        RandomSpeaking = False
    else:
        resultado = GetHistoryAIResponse(resultado)
        print(resultado)
        RandomSpeaking = True

Overriding of current TracerProvider is not allowed


Girafa


Overriding of current TracerProvider is not allowed


Em uma vasta savana africana, onde o sol brilhava intensamente e as árvores se erguiam como torres verdes, vivia uma girafa chamada Gigi. Gigi era uma girafa curiosa e sonhadora, sempre em busca de novas aventuras. Um dia, enquanto caminhava entre as acácias, ela ouviu um rumor vindo de um grupo de animais reunidos em um círculo. Intrigada, Gigi se aproximou e viu que todos estavam falando sobre um misterioso lago que, segundo rumores, tinha o poder de realizar desejos.

Movida pela curiosidade, Gigi decidiu que precisava encontrar esse lago mágico. Com seu longo pescoço, ela conseguia avistar as paisagens ao longe e, após horas de caminhada, avistou uma bela água cristalina refletindo o céu azul. Ao chegar ao lago, Gigi ficou maravilhada com a beleza do lugar. As flores ao redor dançavam com a brisa e o som da água era como uma melodia suave.

Gigi se aproximou da margem e, olhando para a água, fez um pedido sincero: "Eu desejo entender melhor os sentimentos dos meus amigos animais." 

Overriding of current TracerProvider is not allowed


...

Overriding of current TracerProvider is not allowed


Em uma remota e gelada região da Antártica, havia um pinguim chamado Pip. Pip era um pinguim diferente dos outros; enquanto seus amigos passavam o dia deslizando pelo gelo e mergulhando nas águas frias em busca de peixes, ele sonhava em explorar o mundo além do horizonte branco. Ele sempre olhava para o céu e imaginava como seria voar como as aves que passavam por ali.

Um dia, enquanto Pip estava em uma de suas reflexões, ele encontrou uma garrafa flutuando no mar. Curioso, ele a puxou para perto e, para sua surpresa, encontrou um mapa dentro dela. O mapa mostrava uma ilha misteriosa, repleta de tesouros e criaturas fantásticas. O coração de Pip disparou de emoção. Ele decidiu que precisava ir até lá, mesmo que isso significasse deixar sua colônia de pinguins.

Com a ajuda de seus amigos, Pip começou a planejar a jornada. Eles construíram uma pequena jangada com pedaços de gelo e madeira que encontraram na praia. No dia da partida, Pip se despediu de sua família e amigos, prometendo v

Overriding of current TracerProvider is not allowed


Abacaxi


KeyboardInterrupt: 

# Tentativa reiterando a ideia inicial

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

In [4]:
# 1) Configura LLM e memória
llm = ChatOpenAI(openai_api_key=api_key, model_name=model_name, temperature=0.3)
memory = ConversationSummaryMemory(llm=llm)

In [5]:
# 2) Função que gera resposta para um “papel” (comprador ou vendedor, ou gerador de palavra/historiador)
def gerar_resposta(role, objetivo, entrada_outro, memory, llm):
    # Carrega o resumo atual
    resumo = memory.load_memory_variables({})["history"]
    # Monta prompt
    prompt = (
        f"Você é {role}.\n"
        f"Seu objetivo: {objetivo}\n\n"
        f"Resumo da conversa até agora:\n{resumo}\n\n"
    )
    if entrada_outro:
        prompt += f"O outro disse: {entrada_outro}\n"
    prompt += "Responda agora:\n"

    # Gera resposta
    resposta = llm.predict(prompt)

    # Salva no histórico
    memory.save_context({"input": entrada_outro or ""}, {"output": resposta})

    return resposta

In [6]:
# 3) Loop de turnos
n_vezes = 6
mensagem_atual = ""
for i in range(n_vezes):
    if i % 2 == 0:
        # Gerador de palavra
        role = "Gerador de Palavras Aleatórias"
        objetivo = "Gerar uma palavra aleatória (animal, objeto, etc). Nunca repita palavras."
    else:
        # Contador de histórias
        role = "Contador de Histórias"
        objetivo = "Continuar a história existente, incorporando a última palavra recebida."
    # Gera a resposta
    resposta = gerar_resposta(role, objetivo, mensagem_atual, memory, llm)
    print(f"{role}: {resposta}\n")
    mensagem_atual = resposta

C:\Users\guoli\AppData\Local\Temp\ipykernel_40148\3692886920.py:16: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = llm.predict(prompt)


Gerador de Palavras Aleatórias: Canguru.

Contador de Histórias: O outro disse: Canguru. 

O animal, com seu jeito curioso e ágil, pulou de um lado para o outro, atraindo a atenção de todos ao redor. As crianças, fascinadas, começaram a imitar os saltos do canguru, rindo e se divertindo. Mas, no meio da brincadeira, um deles notou algo brilhante no chão. 

"Olhem!", gritou, apontando para o objeto. Era uma pequena pedra preciosa, que refletia a luz do sol em cores vibrantes. O grupo parou imediatamente, intrigado. O que aquela pedra poderia significar? E por que estava ali, em meio à diversão? 

Enquanto todos se aproximavam, um velho sábio que passava pela trilha parou e disse: "Essa pedra é um símbolo de sorte, mas também de responsabilidade. O que vocês farão com ela pode mudar o destino de todos." 

As crianças se entreolharam, sentindo o peso daquelas palavras. O que fazer com a sorte que acabavam de encontrar?

Gerador de Palavras Aleatórias: Elefante.

Contador de Histórias: Enq

# Codigo com vendedor e comprador

In [19]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4")

# Modelo
llm = ChatOpenAI(
    openai_api_key=api_key,
    model_name=model_name,
    temperature=0.3
)

# Memória resumida
memory = ConversationSummaryMemory(llm=llm)

def gerar_resposta(role, objetivo, fala_anterior, memory, turno_atual, turno_total):
    """
    Gera a resposta de um "agente" (Buyer ou Seller) usando a memória resumida.

    role: "Buyer" ou "Seller"
    objetivo: a descrição do que esse papel deve fazer
    fala_anterior: última fala recebida
    memory: ConversationSummaryMemory
    turno_atual, turno_total: controle para forçar decisão no final
    """
    resumo = memory.load_memory_variables({})["history"]

    prompt = f"""
Você é o {role}.
Você e o outro interlocutor estão numa loja de roupas em um shopping, conversando presencialmente.
Seu objetivo: {objetivo}

Resumo da conversa até agora:
{resumo}

A última fala do outro interlocutor foi:
{fala_anterior}

INSTRUÇÕES IMPORTANTES:
1. Mantenha o tom de voz e contexto de uma conversa presencial em loja de shopping.
2. Não assuma o papel do outro. (Se você é Buyer, aja como comprador; se é Seller, aja como vendedor.)
3. Se você for Buyer, faça perguntas, negocie preço, demonstre interesse ou não na compra.
4. Se você for Seller, ofereça camisas, apresente preços, tente fechar a venda.
5. Há no total {turno_total} turnos na conversa. Estamos no turno {turno_atual}.
6. No ÚLTIMO turno (turno {turno_total}), a conversa DEVE ter um resultado concreto:
   - Se for o Buyer falando, ele deve decidir definitivamente se COMPRA ou NÃO COMPRA.
   - NÃO é permitido responder "Vou experimentar", "Vou pensar depois" ou "Talvez eu volte". 
     Isso não é um desfecho definitivo.
   - Se for o Seller falando no último turno e o Buyer não tiver se decidido claramente, o Seller interpreta isso como desistência e encerra amigavelmente.
7. Não se identifique como modelo de IA. Você é uma pessoa real no shopping.

Responda agora (estamos no turno {turno_atual} de {turno_total}):
""".strip()

    resposta = llm.predict(prompt)

    # Salva no histórico da memória
    memory.save_context(
        {"input": fala_anterior},
        {"output": resposta}
    )

    return resposta

def main():
    buyer_role = "Buyer"
    buyer_goal = (
        "Você deseja comprar uma camisa branca para um casamento. "
        "No último turno, você deve dizer claramente se vai comprar ou não."
    )

    seller_role = "Seller"
    seller_goal = (
        "Você trabalha numa loja e quer vender camisas brancas. "
        "Tente convencer o comprador, mas respeite a decisão final."
    )

    # Usamos 5 turnos para que o 5º (último) seja do Buyer, 
    # que então PRECISA decidir (compra ou não).
    turno_total = 5

    fala_anterior = ""

    for turno_atual in range(1, turno_total + 1):
        if turno_atual % 2 == 1:
            # Buyer fala em turnos 1, 3, 5
            role = buyer_role
            objetivo = buyer_goal
        else:
            # Seller fala em turnos 2, 4
            role = seller_role
            objetivo = seller_goal

        resposta = gerar_resposta(role, objetivo, fala_anterior, memory, turno_atual, turno_total)
        print(f"{role}: {resposta}\n")
        fala_anterior = resposta

if __name__ == "__main__":
    main()


Buyer: Oi, tudo bem? Estou procurando uma camisa branca para um casamento que vou. Você tem alguma opção que possa me mostrar?

Seller: Oi! Tudo bem, sim! Temos algumas opções de camisas brancas que podem ser perfeitas para o seu casamento. Aqui estão algumas delas. 

Esta aqui é uma camisa de algodão, bem leve e confortável, ideal para eventos. O preço é R$ 149,90. E temos também uma opção com um tecido um pouco mais sofisticado, que custa R$ 199,90. O que você acha? Alguma dessas opções parece interessante para você?

Buyer: Oi! Estou bem, obrigado! Olhando as opções, a camisa de algodão parece bem confortável e o preço está bom. Mas me diga, essa opção de tecido mais sofisticado, o que a torna diferente? É mais formal?

Seller: Claro! A camisa de tecido mais sofisticado é feita de uma mistura de poliéster e algodão, o que a torna mais leve e com um caimento mais elegante. Ela tem um acabamento que dá um brilho sutil, ideal para ocasiões mais formais, como um casamento. Além disso, a